In [ ]:
'''code carried over from L1_fits-handling.ipynb'''

# open fits file and extract data
from astropy.io import fits
fits_file = fits.open('ngc1261.fits')
image_data = fits_file[0].data

# define section1 (the portion of the image that we will work with)
section1 = image_data[2250:2650, 5350:5950]

# plot section 1
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
plt.figure()
plt.imshow(section1, origin = 'lower', cmap = 'Greys', norm = LogNorm())
plt.colorbar()
plt.show()

In [ ]:
'''find mean, median, and standard deviation'''

from astropy.stats import sigma_clipped_stats
mean, median, std = sigma_clipped_stats(section1,sigma=3.0)
# print data
print((mean,median,std))

In [ ]:
'''Use DAOStarFinder to detect stars'''

from photutils.detection import DAOStarFinder

# find stars that have a full-width-half-maximum of about 3 pixels and
# are at least 5 sigma above the background level
daofind = DAOStarFinder(fwhm = 3.0, threshold = 5.0*std)

# define table named "sources"
sources = daofind(section1 - median)

# format sources table
for col in sources.colnames:
    if col not in ('id', 'npix'):
        sources[col].info.format = '%.2f'

# print table (make sure to use 'p-print' not just 'print')
sources.pprint(max_width = 76)

In [ ]:
'''visualize detected stars with matplotlib'''

import numpy as np
import matplotlib.pyplot as pyplot
from matplotlib.colors import LogNorm
from photutils.aperture import CircularAperture

# define star positions based on sources table, then draw 
# apertures around each position (radius 15) and plot data
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r = 15.0)
plt.imshow(section1, cmap = 'Greys', origin = 'lower', norm = LogNorm(), interpolation = 'nearest')

# draw apertures. apertures.plot command takes arguments (color, line-width, and opacity (alpha))
apertures.plot(color = 'blue', lw = 1.5, alpha = 0.5)

In [ ]:
'''Mask regions with bright stars'''

In [ ]:
'''use find_peaks to detect stars'''

from astropy.stats import sigma_clipped_stats
from photutils.detection import find_peaks

# define mean, median, and std
mean, median, std = sigma_clipped_stats(section1, sigma = 3.0)

# find stars that are at least 5 sigma above background and 
# separated by at least 5 pixels
threshold = median + (5.0*std)

# define, format, and print table of peak values
tbl = find_peaks(section1, threshold, box_size = 11)
tbl['peak_value'].info.format = '%8g'
print(tbl)

In [ ]:
'''visualize locations of detected peaks with matplotlib'''

import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import LogNorm
from photutils.aperture import CircularAperture

# define positions based on peak table and plot data
postitions = np.transpose((tbl['x_peak'], tbl['ypeak']))
plt.imshow(section1, cmap = 'Greys', origin = 'lower', norm = LogNorm(), interpolation = 'nearest')

# plot apertures, but change color to distinguish from DAOStarFinder data
apertures.plot(color = 'red', lw = 1.5, alpha = 0.5)

In [ ]:
'''Mask regions with bright stars'''